In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
n_gpus = len(os.environ['CUDA_VISIBLE_DEVICES'].split(','))
model_name = 'ception_8classifier_trained'
trainval_split_ratio = 0.88
train_batch_size =16

import keras.backend as K
import tensorflow as tf

# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.7)
# sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

Using TensorFlow backend.


# 导入必要的库

In [2]:
import numpy as np
import pandas as pd
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.applications import *
from keras.regularizers import l2

from keras.preprocessing.image import *

import matplotlib.pyplot as plt
import random
import os
import cv2
from tqdm import tqdm
from glob import glob
import multiprocessing

from sklearn.model_selection import train_test_split
from collections import Counter
from keras import backend as K
from keras.utils import multi_gpu_model

from IPython.display import display, Image

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# 读取数据集

In [3]:
df = pd.read_csv('../train/Annotations/train.csv', header=None)
df.columns = ['filename', 'label_name', 'label']
df = df.sample(frac=1).reset_index(drop=True) # shuffle

df.label_name = df.label_name.str.replace('_labels', '')

display(df.head())
c = Counter(df.label_name)
c

,filename,label_name,label
0,Images/skirt_length_labels/b25a30c958f2c730829...,skirt_length,nnnnny
1,Images/coat_length_labels/7f8906613615aa828171...,coat_length,nynnnnnn
2,Images/sleeve_length_labels/2a0928a3ad9b3d2287...,sleeve_length,nnynnnnnn
3,Images/skirt_length_labels/ccc4dc71ddf9dc7f404...,skirt_length,nnnynn
4,Images/sleeve_length_labels/333f6914602312723f...,sleeve_length,nymnnnnnn


Counter({'coat_length': 11320,
         'collar_design': 8393,
         'lapel_design': 7034,
         'neck_design': 5696,
         'neckline_design': 17148,
         'pant_length': 7460,
         'skirt_length': 19333,
         'sleeve_length': 13299})

In [4]:
label_count = dict([(x, len(df[df.label_name == x].label.values[0])) for x in c.keys()])
label_names = list(label_count.keys())
display(label_count)

{'coat_length': 8,
 'collar_design': 5,
 'lapel_design': 5,
 'neck_design': 5,
 'neckline_design': 10,
 'pant_length': 6,
 'skirt_length': 6,
 'sleeve_length': 9}

## 生成 y

In [5]:
fnames = df['filename'].values
#width = 331
width = 299
n = len(df)
y = [np.zeros((n, label_count[x])) for x in label_count.keys()]
for i in range(n):
    label_name = df.label_name[i]
    label = df.label[i]
    y[label_names.index(label_name)][i, label.find('y')] = 1

## 读取图片

In [6]:
def f(index):
    return index, cv2.resize(cv2.imread('../train/'+fnames[index]), (width, width))

X = np.zeros((n, width, width, 3), dtype=np.uint8)
with multiprocessing.Pool(30) as pool:
    with tqdm(pool.imap_unordered(f, range(n)), total=n) as pbar:
        for i, img in pbar:
            X[i] = img[:,:,::-1]

100%|██████████| 89683/89683 [18:15<00:00, 81.88it/s] 


In [7]:
n_train = int(n*trainval_split_ratio)

X_train = X[:n_train]
X_valid = X[n_train:]
y_train = [x[:n_train] for x in y]
y_valid = [x[n_train:] for x in y]

In [8]:
def display_images(imgs, w=8, h=4, figsize=(24, 12)):
    plt.figure(figsize=figsize)
    for i in range(w*h):
        plt.subplot(h, w, i+1)
        plt.imshow(imgs[i])

In [9]:
class Generator():
    def __init__(self, X, y, batch_size=32, aug=False):
        def generator():
            idg = ImageDataGenerator(horizontal_flip=True,
                                     rotation_range=15,
                                     zoom_range=0.15)
            while True:
                for i in range(0, len(X), batch_size):
                    X_batch = X[i:i+batch_size].copy()
                    y_barch = [x[i:i+batch_size] for x in y]
                    if aug:
                        for j in range(len(X_batch)):
                            X_batch[j] = idg.random_transform(X_batch[j])
                    yield X_batch, y_barch
        self.generator = generator()
        self.steps = len(X) // batch_size + 1

In [10]:
gen_train = Generator(X_train, y_train, batch_size=train_batch_size, aug=True)

# 数据集探索

## 类别分布

In [11]:
# plt.figure(figsize=(26, 14))
# for i in range(8):
#     plt.subplot(2, 4, i+1)
#     counts = Counter(y[i].argmax(axis=-1)[np.where(y[i].any(axis=-1))])
#     pd.Series(counts).plot('bar')

### 抽样可视化

In [12]:
# plt.figure(figsize=(26, 14))
# w = 8
# h = 4
# for i in range(w*h):
#     plt.subplot(h, w, i+1)
#     index = np.random.randint(n)
#     plt.title(str([y[x][index].argmax() if y[x][index].any() else -1 for x in range(8) ]))
#     plt.imshow(X[index])
#     plt.axis('off')

# 搭建模型并训练

## 搭建模型

In [13]:
def acc(y_true, y_pred):
    index = tf.reduce_any(y_true > 0.5, axis=-1)
    res = tf.equal(tf.argmax(y_true, axis=-1), tf.argmax(y_pred, axis=-1))
    index = tf.cast(index, tf.float32)
    res = tf.cast(res, tf.float32)
    return tf.reduce_sum(res * index) / (tf.reduce_sum(index) + 1e-7)

In [14]:
base_model = InceptionResNetV2(weights='imagenet', input_shape=(width, width, 3), include_top=False, pooling='avg')
#InceptionResNetV2,Xception
input_tensor = Input((width, width, 3))
x = input_tensor
x = Lambda(nasnet.preprocess_input)(x)
x = base_model(x)
x = Dropout(0.5)(x)
x = [Dense(count, activation='softmax', name=name)(x) for name, count in label_count.items()]

model = Model(input_tensor, x)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

from IPython.display import SVG
#from keras.utils.vis_utils import model_to_dot, plot_model

#plot_model(model, show_shapes=True, to_file='model_simple.png')
#SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

In [15]:
#model2 = multi_gpu_model(model, n_gpus)
model2 = model

In [16]:
opt = SGD(1e-2, momentum=0.9, nesterov=True, decay=0.01/40)

In [17]:
model2.compile(optimizer=opt, loss='categorical_crossentropy', metrics=[acc])
model2.fit_generator(gen_train.generator, steps_per_epoch=gen_train.steps, epochs=4, validation_data=(X_valid, y_valid))

Epoch 1/4
4933/4933 [==============================] - 2401s 487ms/step - loss: 0.8771 - skirt_length_loss: 0.1390 - coat_length_loss: 0.1130 - sleeve_length_loss: 0.1350 - lapel_design_loss: 0.0679 - neckline_design_loss: 0.2116 - collar_design_loss: 0.0830 - pant_length_loss: 0.0589 - neck_design_loss: 0.0685 - skirt_length_acc: 0.7480 - coat_length_acc: 0.5936 - sleeve_length_acc: 0.6009 - lapel_design_acc: 0.4883 - neckline_design_acc: 0.5939 - collar_design_acc: 0.5248 - pant_length_acc: 0.5490 - neck_design_acc: 0.3722 - val_loss: 0.5512 - val_skirt_length_loss: 0.0928 - val_coat_length_loss: 0.0634 - val_sleeve_length_loss: 0.0747 - val_lapel_design_loss: 0.0452 - val_neckline_design_loss: 0.1280 - val_collar_design_loss: 0.0622 - val_pant_length_loss: 0.0349 - val_neck_design_loss: 0.0500 - val_skirt_length_acc: 0.7626 - val_coat_length_acc: 0.5751 - val_sleeve_length_acc: 0.6176 - val_lapel_design_acc: 0.3922 - val_neckline_design_acc: 0.6700 - val_collar_design_acc: 0.4525 - 

In [18]:
model2.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=[acc])
model2.fit_generator(gen_train.generator, steps_per_epoch=gen_train.steps, epochs=4, validation_data=(X_valid, y_valid))

Epoch 1/4
4933/4933 [==============================] - 2258s 458ms/step - loss: 0.3937 - skirt_length_loss: 0.0625 - coat_length_loss: 0.0549 - sleeve_length_loss: 0.0563 - lapel_design_loss: 0.0268 - neckline_design_loss: 0.0909 - collar_design_loss: 0.0381 - pant_length_loss: 0.0300 - neck_design_loss: 0.0341 - skirt_length_acc: 0.8769 - coat_length_acc: 0.7446 - sleeve_length_acc: 0.7919 - lapel_design_acc: 0.6406 - neckline_design_acc: 0.8154 - collar_design_acc: 0.6726 - pant_length_acc: 0.6510 - neck_design_acc: 0.5199 - val_loss: 0.4851 - val_skirt_length_loss: 0.0793 - val_coat_length_loss: 0.0730 - val_sleeve_length_loss: 0.0613 - val_lapel_design_loss: 0.0402 - val_neckline_design_loss: 0.1059 - val_collar_design_loss: 0.0479 - val_pant_length_loss: 0.0316 - val_neck_design_loss: 0.0460 - val_skirt_length_acc: 0.8571 - val_coat_length_acc: 0.7006 - val_sleeve_length_acc: 0.7813 - val_lapel_design_acc: 0.6064 - val_neckline_design_acc: 0.7913 - val_collar_design_acc: 0.6599 - 

In [19]:
model2.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', metrics=[acc])
model2.fit_generator(gen_train.generator, steps_per_epoch=gen_train.steps, epochs=4, validation_data=(X_valid, y_valid))

Epoch 1/4
4933/4933 [==============================] - 2273s 461ms/step - loss: 0.1365 - skirt_length_loss: 0.0182 - coat_length_loss: 0.0210 - sleeve_length_loss: 0.0199 - lapel_design_loss: 0.0077 - neckline_design_loss: 0.0338 - collar_design_loss: 0.0125 - pant_length_loss: 0.0107 - neck_design_loss: 0.0126 - skirt_length_acc: 0.9541 - coat_length_acc: 0.8355 - sleeve_length_acc: 0.8822 - lapel_design_acc: 0.7073 - neckline_design_acc: 0.9098 - collar_design_acc: 0.7609 - pant_length_acc: 0.7215 - neck_design_acc: 0.6055 - val_loss: 0.3568 - val_skirt_length_loss: 0.0343 - val_coat_length_loss: 0.0540 - val_sleeve_length_loss: 0.0487 - val_lapel_design_loss: 0.0299 - val_neckline_design_loss: 0.0854 - val_collar_design_loss: 0.0451 - val_pant_length_loss: 0.0277 - val_neck_design_loss: 0.0318 - val_skirt_length_acc: 0.9337 - val_coat_length_acc: 0.7705 - val_sleeve_length_acc: 0.8316 - val_lapel_design_acc: 0.6522 - val_neckline_design_acc: 0.8377 - val_collar_design_acc: 0.6938 - 

In [20]:
model2.compile(optimizer=Adam(1e-6), loss='categorical_crossentropy', metrics=[acc])
model2.fit_generator(gen_train.generator, steps_per_epoch=gen_train.steps, epochs=1, validation_data=(X_valid, y_valid))

Epoch 1/1
4933/4933 [==============================] - 2278s 462ms/step - loss: 0.0463 - skirt_length_loss: 0.0055 - coat_length_loss: 0.0069 - sleeve_length_loss: 0.0068 - lapel_design_loss: 0.0022 - neckline_design_loss: 0.0124 - collar_design_loss: 0.0039 - pant_length_loss: 0.0041 - neck_design_loss: 0.0045 - skirt_length_acc: 0.9723 - coat_length_acc: 0.8706 - sleeve_length_acc: 0.9100 - lapel_design_acc: 0.7247 - neckline_design_acc: 0.9479 - collar_design_acc: 0.7853 - pant_length_acc: 0.7411 - neck_design_acc: 0.6346 - val_loss: 0.3925 - val_skirt_length_loss: 0.0349 - val_coat_length_loss: 0.0571 - val_sleeve_length_loss: 0.0559 - val_lapel_design_loss: 0.0310 - val_neckline_design_loss: 0.0955 - val_collar_design_loss: 0.0511 - val_pant_length_loss: 0.0311 - val_neck_design_loss: 0.0358 - val_skirt_length_acc: 0.9356 - val_coat_length_acc: 0.7752 - val_sleeve_length_acc: 0.8293 - val_lapel_design_acc: 0.6493 - val_neckline_design_acc: 0.8381 - val_collar_design_acc: 0.7013 - 

# 计算验证集准确率

In [21]:
model.save('B_model_%s.h5' % model_name)

In [22]:
y_pred = model2.predict(X_valid, batch_size=128, verbose=1)
a = np.array([x.any(axis=-1) for x in y_valid]).T.astype('uint8')
b = [np.where((a == np.eye(8)[x]).all(axis=-1))[0] for x in range(8)]
for c in range(8):
    y_pred2 = y_pred[c][b[c]].argmax(axis=-1)
    y_true2 = y_valid[c][b[c]].argmax(axis=-1)
    print(label_names[c], (y_pred2 == y_true2).mean())

10762/10762 [==============================] - 116s 11ms/step
skirt_length 0.961618690029
coat_length 0.878587196468
sleeve_length 0.894736842105
lapel_design 0.890840652447
neckline_design 0.871373307544
collar_design 0.862403100775
pant_length 0.889411764706
neck_design 0.843137254902


In [23]:
counts = Counter(df.label_name)
counts

Counter({'coat_length': 11320,
         'collar_design': 8393,
         'lapel_design': 7034,
         'neck_design': 5696,
         'neckline_design': 17148,
         'pant_length': 7460,
         'skirt_length': 19333,
         'sleeve_length': 13299})

In [24]:
s = 0
n = 0
for c in range(8):
    y_pred2 = y_pred[c][b[c]].argmax(axis=-1)
    y_true2 = y_valid[c][b[c]].argmax(axis=-1)
    s += counts[label_names[c]] * (y_pred2 == y_true2).mean()
    n += counts[label_names[c]]
print(s / n)

0.895597157566


# 在测试集上预测

In [25]:
df_test = pd.read_csv('../test/z_rank/Tests/question.csv', header=None)
df_test.columns = ['filename', 'label_name', 'label']

fnames_test = df_test.filename

n_test = len(df_test)
df_test.head()

,filename,label_name,label
0,Images/collar_design_labels/faad3490a16c7f3d4f...,collar_design_labels,?
1,Images/collar_design_labels/0b2b4254f35ce3a41a...,collar_design_labels,?
2,Images/collar_design_labels/7f2be608e06f804dd5...,collar_design_labels,?
3,Images/collar_design_labels/4b09d4dca80caac42e...,collar_design_labels,?
4,Images/collar_design_labels/de91f00a05e84d7239...,collar_design_labels,?


In [26]:
def f(index):
    return index, cv2.resize(cv2.imread('../test/z_rank/'+fnames_test[index]), (width, width))

X_test = np.zeros((n_test, width, width, 3), dtype=np.uint8)
with multiprocessing.Pool(12) as pool:
    with tqdm(pool.imap_unordered(f, range(n_test)), total=n_test) as pbar:
        for i, img in pbar:
            X_test[i] = img[:,:,::-1]

100%|██████████| 15042/15042 [01:46<00:00, 141.42it/s]


In [27]:
y_pred = model2.predict(X_test, batch_size=128, verbose=1)

15042/15042 [==============================] - 162s 11ms/step


In [28]:
for i in range(n_test):
    problem_name = df_test.label_name[i].replace('_labels', '')
    problem_index = label_names.index(problem_name)
    probs = y_pred[problem_index][i]
    df_test.label[i] = ';'.join(np.char.mod('%.8f', probs))

In [29]:
fname_csv = 'B_test_%sv2.csv' % model_name
df_test.to_csv(fname_csv, index=None, header=None)